# Tutorial using custom genome on the PBMC bulk ATAC-seq

In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="anndata")
import scprinter as scp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
import pandas as pd
import numpy as np
import os
import pickle
import torch
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
from scanpy.plotting.palettes import zeileis_28
from tqdm.contrib.concurrent import *
from tqdm.auto import *
import anndata
import scanpy as sc
import json
import csv
import re
from sklearn.preprocessing import OneHotEncoder

/net/talisker/home/benos/mae117/.conda/envs/tmm/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/net/talisker/home/benos/mae117/.conda/envs/tmm/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. 

In [2]:
def density_plot(x, y, logx=False, logy=False):
    import numpy as np
    import seaborn as sns
    import matplotlib.pyplot as plt
    from scipy.stats import gaussian_kde
    # 2D histogram binning to estimate density
    bins = 10000  # tune this for resolution/performance
    hist, xedges, yedges = np.histogram2d(x, y, bins=bins)

    # Assign each point a density based on its bin
    x_bin = np.searchsorted(xedges, x, side='right') - 1
    y_bin = np.searchsorted(yedges, y, side='right') - 1

    # Clip out-of-bound indices
    mask = (x_bin >= 0) & (x_bin < bins) & (y_bin >= 0) & (y_bin < bins)
    density = np.zeros_like(x, dtype=float)
    density[mask] = np.log10(hist[x_bin[mask], y_bin[mask]])

    # Sort by density for better visual effect
    idx = density.argsort()
    x, y, density = x[idx], y[idx], density[idx]
    x = np.log10(x) if logx else x
    y = np.log10(y) if logy else y
    sns.scatterplot(x=x,y=y,hue=density,linewidth=0, s=1, palette='viridis')

In [3]:
main_dir = '/net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man'
work_dir = f'{main_dir}/seq2print'
if not os.path.exists(work_dir):
    os.system("mkdir -p " + work_dir)
frag_dir = f'{main_dir}/fragments'
if not os.path.exists(frag_dir):
    os.system("mkdir -p " + frag_dir)

In [4]:
# Download the fragments files of bulk samples from our Zenodo repository https://zenodo.org/records/14866808 using the code below
samples = ["Bcell_0", "Bcell_1", "Monocyte_0", "Monocyte_1", "Tcell_0", "Tcell_1"]
for sample in samples:
    link_prefix = "https://zenodo.org/records/14866808/files/PBMC_bulk_ATAC_tutorial_"
    link_suffix = "_frags.tsv.gz?download=1"
    os.system(f"wget -O {frag_dir}/{sample}_frags.tsv.gz {link_prefix}{sample}{link_suffix}")

--2025-10-30 10:24:15--  https://zenodo.org/records/14866808/files/PBMC_bulk_ATAC_tutorial_Bcell_0_frags.tsv.gz?download=1
Resolving zenodo.org (zenodo.org)... 188.185.43.153, 137.138.52.235, 188.185.48.75, ...
Connecting to zenodo.org (zenodo.org)|188.185.43.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 92267715 (88M) [application/octet-stream]
Saving to: ‘/net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/fragments/Bcell_0_frags.tsv.gz’

     0K .......... .......... .......... .......... ..........  0%  216K 6m57s
    50K .......... .......... .......... .......... ..........  0%  528K 4m54s
   100K .......... .......... .......... .......... ..........  0%  539K 4m11s
   150K .......... .......... .......... .......... ..........  0% 9.85M 3m11s
   200K .......... .......... .......... .......... ..........  0%  540K 3m6s
   250K .......... .......... .......... .......... ..........  0% 13.4M 2m36s
   300K .......... .......... ..

In [5]:
# Get the fragments files for individual samples
# Note: the fragments files should have 4 columns: chromosome, start, end, and barcode.
# Barcode should just be sample name, which means all fragments in the same fragments file have the same barcode.
frag_files = os.listdir(frag_dir)
frag_files = [i for i in frag_files if re.search("frags.tsv.gz", i) is not None]
frag_files = sorted([os.path.join(frag_dir, i) for i in frag_files])
samples = ["_".join(re.split("[/_\\.]", i)[11:13]) for i in frag_files]
samples

['Bcell_0', 'Bcell_1', 'Monocyte_0', 'Monocyte_1', 'Tcell_0', 'Tcell_1']

# Create scprinter & preprocessing

In [6]:
printer = scp.pp.import_fragments(
                        path_to_frags=frag_files,
                        barcodes=[None] * len(frag_files), # This loads individual fragments files and combine them
                        savename=os.path.join(work_dir, 'PBMC_bulkATAC_scprinter.h5ad'),
                        genome=scp.genome.hg38,
                        min_num_fragments=1000, min_tsse=7,
                        sorted_by_barcode=False,
                        low_memory=False,
                        )

Multiple fragments files detected, it is suggested to provide sample names to avoid barcode collision
You are now using the beta auto_detect_shift function, this overwrites the plus_shift and minus_shift you provided
If you believe the auto_detect_shift is wrong, please set auto_detect_shift=False


Importing fragments:   0%|          | 0/6 [00:00<?, ?it/s]

Detecting the shift in the paired end fragments file
If you think the above message is wrong, please check the input file format
Minimum MSE is 0.0005501932766544962, shift detected
Minimum MSE is 0.0005658050675804094, shift detected
detected plus_shift and minus_shift are 4 -5 for /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/fragments/Bcell_0_frags.tsv.gz
Detecting the shift in the paired end fragments file
If you think the above message is wrong, please check the input file format
Minimum MSE is 0.0005967584874449537, shift detected
Minimum MSE is 0.0006268005023257401, shift detected
detected plus_shift and minus_shift are 4 -5 for /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/fragments/Bcell_1_frags.tsv.gz


/net/talisker/home/benos/mae117/.conda/envs/tmm/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/net/talisker/home/benos/mae117/.conda/envs/tmm/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. 

Detecting the shift in the paired end fragments file
If you think the above message is wrong, please check the input file format
Minimum MSE is 0.00016154011093750005, shift detected
Minimum MSE is 0.0001239263105125, shift detected
detected plus_shift and minus_shift are 4 -5 for /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/fragments/Monocyte_0_frags.tsv.gz


...

/net/talisker/home/benos/mae117/.conda/envs/tmm/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/net/talisker/home/benos/mae117/.conda/envs/tmm/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. 

Detecting the shift in the paired end fragments file
If you think the above message is wrong, please check the input file format
Minimum MSE is 0.0009838926211303955, shift detected
Minimum MSE is 0.00102080718604669, shift detected
detected plus_shift and minus_shift are 4 -5 for /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/fragments/Tcell_1_frags.tsv.gz


  0%|          | 0/6 [00:00<?, ?it/s]

/net/talisker/home/benos/mae117/.conda/envs/tmm/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/net/talisker/home/benos/mae117/.conda/envs/tmm/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. 

start transferring insertions


In [7]:
print (printer.insertion_file.obs_names[:])

['Bcell_0', 'Bcell_1', 'Monocyte_0', 'Monocyte_1', 'Tcell_0', 'Tcell_1']


In [8]:
# Rename barcodes to sample IDs if needed.
printer.insertion_file.obs_names = samples

# Call peaks on the fragments in a preset for training seq2PRINT model

In [10]:
# Call peaks, this set of peaks are recommended to train seq2PRINT model
# scp.pp.call_peaks(printer=printer,
#                   frag_file=frag_files,
#                   cell_grouping=[None], # here we call peaks on the cells that are included in the final analyses
#                   group_names=['all'],
#                   preset='seq2PRINT',
#                   overwrite=False)

In [11]:
# Call peaks using chromvar preset, this set of peak are recommended to be use as cell x peak for scATAC-seq data, or analysis
# scp.pp.call_peaks(printer=printer,
#                   frag_file=frag_files,
#                   cell_grouping=[None], # here we call peaks on the cells that are included in the final analyses
#                   group_names=['chromvar_all'],
#                   preset='chromvar',
#                   overwrite=False)

There is an existing issue with scPrinter and the calling of MACS2 for peak calling. This can be mitigated, at least in my case, by simply running the same command, but with MACS3 [1](https://github.com/buenrostrolab/scPrinter/issues/45), [2](https://github.com/buenrostrolab/scPrinter/issues/14), [3](https://github.com/buenrostrolab/scPrinter/issues/12), [4](https://github.com/buenrostrolab/scPrinter/issues/11), [5](https://github.com/buenrostrolab/scPrinter/issues/10)

```
macs2 callpeak --nomodel -t /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/fragments/Bcell_0_frags.tsv.gz /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/fragments/Bcell_1_frags.tsv.gz /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/fragments/Monocyte_0_frags.tsv.gz /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/fragments/Monocyte_1_frags.tsv.gz /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/fragments/Tcell_0_frags.tsv.gz /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/fragments/Tcell_1_frags.tsv.gz --outdir /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print/PBMC_bulkATAC_scprinter_supp/macs2 -n all -f BEDPE --nolambda --keep-dup all --call-summits --nomodel -B --SPMR --shift 75 --extsize 150 -p 0.01
```

to

```
macs3 callpeak --nomodel -t /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/fragments/Bcell_0_frags.tsv.gz /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/fragments/Bcell_1_frags.tsv.gz /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/fragments/Monocyte_0_frags.tsv.gz /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/fragments/Monocyte_1_frags.tsv.gz /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/fragments/Tcell_0_frags.tsv.gz /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/fragments/Tcell_1_frags.tsv.gz --outdir /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print/PBMC_bulkATAC_scprinter_supp/macs2 -n all -f BEDPE --nolambda --keep-dup all --call-summits --nomodel -B --SPMR --shift 75 --extsize 150 -p 0.01
```

and

```
macs2 callpeak --nomodel -t /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/fragments/Bcell_0_frags.tsv.gz /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/fragments/Bcell_1_frags.tsv.gz /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/fragments/Monocyte_0_frags.tsv.gz /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/fragments/Monocyte_1_frags.tsv.gz /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/fragments/Tcell_0_frags.tsv.gz /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/fragments/Tcell_1_frags.tsv.gz --outdir /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print/PBMC_bulkATAC_scprinter_supp/macs2 -n chromvar_all -f BEDPE --nolambda --keep-dup all --call-summits --nomodel -B --SPMR --shift 75 --extsize 150 -q 0.01
```

to 

```
macs3 callpeak --nomodel -t /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/fragments/Bcell_0_frags.tsv.gz /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/fragments/Bcell_1_frags.tsv.gz /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/fragments/Monocyte_0_frags.tsv.gz /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/fragments/Monocyte_1_frags.tsv.gz /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/fragments/Tcell_0_frags.tsv.gz /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/fragments/Tcell_1_frags.tsv.gz --outdir /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print/PBMC_bulkATAC_scprinter_supp/macs2 -n chromvar_all -f BEDPE --nolambda --keep-dup all --call-summits --nomodel -B --SPMR --shift 75 --extsize 150 -q 0.01
```

In [15]:
!alias macs2='macs3'

In [16]:
# Call peaks, this set of peaks are recommended to train seq2PRINT model
scp.pp.call_peaks(printer=printer,
                  frag_file=frag_files,
                  cell_grouping=[None], # here we call peaks on the cells that are included in the final analyses
                  group_names=['all'],
                  preset='seq2PRINT',
                  overwrite=False)

# Fetched the cleaned peaks, save, it will be used in the next step
cleaned_peaks = pd.DataFrame(printer.uns["peak_calling"]['all_cleaned'][:])
cleaned_peaks.to_csv(f'{work_dir}/seq2print_cleaned_narrowPeak.bed',
                     sep='\t', header=False, index=False)

continue


100%|█████████████████████████████████████| 5.87k/5.87k [00:00<00:00, 3.95MB/s]
Decompressing '/net/talisker/home/benos/mae117/.cache/scprinter/hg38-blacklist.v2.bed.gz' to '/net/talisker/home/benos/mae117/.cache/scprinter/hg38-blacklist.v2.bed.gz.decomp' using method 'gzip'.


Reading in peak summit file(s):
NOTE: Assuming all start coordinates are 0-based ..

Padding peak summits by: 500 bp on either side for
Removing peaks overlapping with blacklisted regions and out of bound peaks based on chromosome sizes ..

Filtering overlapping peaks based on peak summit score ..
round: 1 299137 peaks unresolved 295927 peaks selected
round: 2 146 peaks unresolved 142 peaks selected
finish clearing
finish sorting
finished summary


/net/talisker/home/benos/mae117/.local/share/Trash/ipykernel_21293/1374238562.py:10: DeprecationWarning: `_import_from_c` is deprecated; use `_import_arrow_from_c` instead. If you are using an extension, please compile it with the latest 'pyo3-polars'
  cleaned_peaks = pd.DataFrame(printer.uns["peak_calling"]['all_cleaned'][:])


In [17]:
# Call peaks using chromvar preset, this set of peak are recommended to be use as cell x peak for scATAC-seq data, or analysis
scp.pp.call_peaks(printer=printer,
                  frag_file=frag_files,
                  cell_grouping=[None], # here we call peaks on the cells that are included in the final analyses
                  group_names=['chromvar_all'],
                  preset='chromvar',
                  overwrite=False)

# Fetched the cleaned peaks, save, it will be used in the next step
cleaned_peaks = pd.DataFrame(printer.uns["peak_calling"]['chromvar_all_cleaned'][:])
cleaned_peaks.to_csv(f'{work_dir}/regions.bed',
                     sep='\t', header=False, index=False)

continue
Reading in peak summit file(s):
NOTE: Assuming all start coordinates are 0-based ..

Padding peak summits by: 400 bp on either side for
Removing peaks overlapping with blacklisted regions and out of bound peaks based on chromosome sizes ..

Filtering overlapping peaks based on peak summit score ..
round: 1 199743 peaks unresolved 128196 peaks selected
round: 2 21086 peaks unresolved 9588 peaks selected
round: 3 4431 peaks unresolved 1534 peaks selected
round: 4 1362 peaks unresolved 393 peaks selected
round: 5 465 peaks unresolved 116 peaks selected
round: 6 183 peaks unresolved 40 peaks selected
round: 7 65 peaks unresolved 18 peaks selected
round: 8 24 peaks unresolved 5 peaks selected
round: 9 6 peaks unresolved 1 peaks selected
round: 10 1 peaks unresolved 1 peaks selected
finish clearing
finish sorting
finished summary


/net/talisker/home/benos/mae117/.local/share/Trash/ipykernel_21293/65003063.py:10: DeprecationWarning: `_import_from_c` is deprecated; use `_import_arrow_from_c` instead. If you are using an extension, please compile it with the latest 'pyo3-polars'
  cleaned_peaks = pd.DataFrame(printer.uns["peak_calling"]['chromvar_all_cleaned'][:])


In [18]:
# we can compare two sets of peaks with different preset
print (pd.DataFrame(printer.uns["peak_calling"]['all_cleaned'][:]))

           0          1          2                3        4        5    6  \
0       chr1     816850     817850      all_peak_54  153.156  153.156  all   
1       chr1     826420     827420     all_peak_55a   22.458   22.458  all   
2       chr1     827037     828037     all_peak_55b    937.1    937.1  all   
3       chr1     828500     829500      all_peak_56  7.18532  7.18532  all   
4       chr1     843661     844661     all_peak_57a  6.46058  6.46058  all   
...      ...        ...        ...              ...      ...      ...  ...   
296064  chrX  155768168  155769168  all_peak_207726  7.18532  7.18532  all   
296065  chrX  155819803  155820803  all_peak_207727   28.839   28.839  all   
296066  chrX  155841026  155842026  all_peak_207728  36.7673  36.7673  all   
296067  chrX  155880773  155881773  all_peak_207729  665.752  665.752  all   
296068  chrX  155972177  155973177  all_peak_207730  2.71106  2.71106  all   

             7  
0        31037  
1        92205  
2         79

/net/talisker/home/benos/mae117/.local/share/Trash/ipykernel_21293/2415760104.py:2: DeprecationWarning: `_import_from_c` is deprecated; use `_import_arrow_from_c` instead. If you are using an extension, please compile it with the latest 'pyo3-polars'
  print (pd.DataFrame(printer.uns["peak_calling"]['all_cleaned'][:]))


In [19]:
print (pd.DataFrame(printer.uns["peak_calling"]['chromvar_all_cleaned'][:]))

           0          1          2                         3        4  \
0       chr1     817200     817500      chromvar_all_peak_35  150.718   
1       chr1     827387     827687     chromvar_all_peak_36b  933.901   
2       chr1     844607     844907     chromvar_all_peak_37d  13.7316   
3       chr1     858011     858311      chromvar_all_peak_38  14.6569   
4       chr1     865660     865960      chromvar_all_peak_40  107.404   
...      ...        ...        ...                       ...      ...   
139887  chrX  155632509  155632809  chromvar_all_peak_130192  50.4941   
139888  chrX  155767443  155767743  chromvar_all_peak_130194  22.5783   
139889  chrX  155820153  155820453  chromvar_all_peak_130195  26.8421   
139890  chrX  155841376  155841676  chromvar_all_peak_130196  34.7131   
139891  chrX  155881123  155881423  chromvar_all_peak_130197  662.749   

              5             6      7  
0       150.718  chromvar_all  25130  
1       933.901  chromvar_all   7310  
2     

/net/talisker/home/benos/mae117/.local/share/Trash/ipykernel_21293/2938188741.py:1: DeprecationWarning: `_import_from_c` is deprecated; use `_import_arrow_from_c` instead. If you are using an extension, please compile it with the latest 'pyo3-polars'
  print (pd.DataFrame(printer.uns["peak_calling"]['chromvar_all_cleaned'][:]))


# Training separate seq2PRINT models for each sample

In [20]:
import json
model_configs = []
if not os.path.exists(os.path.join(work_dir, 'configs')):
    os.makedirs(os.path.join(work_dir, 'configs'))
for sample in samples:
    # For exploratory analyses we run only one fold. For publication results you can run all 5 fold for slight improvement in accuracy.
    # If you want to run all five folds, add a for-loop layer "for fold in range(5)". This way each sample will have 5 models trained.
    fold=0
    model_config= scp.tl.seq_model_config(printer,
                                     region_path=f'{work_dir}/seq2print_cleaned_narrowPeak.bed',
                                     cell_grouping=[sample],
                                     group_names=sample,
                                     genome=printer.genome,
                                     fold=fold,
                                     overwrite_bigwig=False,
                                     model_name='PBMC_bulkATAC_' + sample,
                                     additional_config={
                                        "notes": "v3",
                                        "tags": ["PBMC_bulkATAC", sample,
                                            f"fold{fold}"]},
                                     path_swap=(work_dir, ''),
                                     config_save_path=f'{work_dir}/configs/PBMC_bulkATAC_{sample}_fold{fold}.JSON')
    model_configs.append(model_config)

Loading insertion profiles
Creating bigwig for Bcell_0


  0%|          | 0/24 [00:00<?, ?it/s]

Creating bigwig for Bcell_1


  0%|          | 0/24 [00:00<?, ?it/s]

Creating bigwig for Monocyte_0


...

  0%|          | 0/24 [00:00<?, ?it/s]

Creating bigwig for Tcell_0


  0%|          | 0/24 [00:00<?, ?it/s]

Creating bigwig for Tcell_1


  0%|          | 0/24 [00:00<?, ?it/s]

In [21]:
for path in ['temp','model']:
    if not os.path.exists(os.path.join(work_dir, path)):
        os.makedirs(os.path.join(work_dir, path))

for sample in samples:
    scp.tl.launch_seq2print(model_config_path=f'{work_dir}/configs/PBMC_bulkATAC_{sample}_fold{fold}.JSON',
                            temp_dir=f'{work_dir}/temp',
                            model_dir=f'{work_dir}/model',
                            data_dir=work_dir,
                            gpus=samples.index(sample),
                            wandb_project='scPrinter_seq_PBMC_bulkATAC', # wandb helps you manage loggins
                            verbose=False,
                            launch=False # launch=True, this command would launch the scripts directly,
                            # otherwise, it will just display the commands, you should copy them and run them.
                           )

CUDA_VISIBLE_DEVICES=0 seq2print_train --config /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print/configs/PBMC_bulkATAC_Bcell_0_fold0.JSON --temp_dir /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print/temp --model_dir /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print/model --data_dir /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print --project scPrinter_seq_PBMC_bulkATAC --enable_wandb
CUDA_VISIBLE_DEVICES=1 seq2print_train --config /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print/configs/PBMC_bulkATAC_Bcell_1_fold0.JSON --temp_dir /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print/temp --model_dir /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print/model --data_dir /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print --project scPrinter_seq_PBMC_bulkATAC -

In [ ]:
!CUDA_VISIBLE_DEVICES=0 seq2print_train --config /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print/configs/PBMC_bulkATAC_Bcell_0_fold0.JSON --temp_dir /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print/temp --model_dir /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print/model --data_dir /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print --project scPrinter_seq_PBMC_bulkATAC --enable_wandb

/net/talisker/home/benos/mae117/.conda/envs/tmm/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/net/talisker/home/benos/mae117/.conda/envs/tmm/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. 

In [ ]:
!CUDA_VISIBLE_DEVICES=0 seq2print_train --config /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print/configs/PBMC_bulkATAC_Bcell_1_fold0.JSON --temp_dir /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print/temp --model_dir /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print/model --data_dir /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print --project scPrinter_seq_PBMC_bulkATAC --enable_wandb

In [ ]:
!CUDA_VISIBLE_DEVICES=0 seq2print_train --config /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print/configs/PBMC_bulkATAC_Monocyte_0_fold0.JSON --temp_dir /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print/temp --model_dir /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print/model --data_dir /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print --project scPrinter_seq_PBMC_bulkATAC --enable_wandb

In [ ]:
!CUDA_VISIBLE_DEVICES=0 seq2print_train --config /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print/configs/PBMC_bulkATAC_Monocyte_1_fold0.JSON --temp_dir /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print/temp --model_dir /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print/model --data_dir /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print --project scPrinter_seq_PBMC_bulkATAC --enable_wandb

In [ ]:
!CUDA_VISIBLE_DEVICES=0 seq2print_train --config /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print/configs/PBMC_bulkATAC_Tcell_0_fold0.JSON --temp_dir /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print/temp --model_dir /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print/model --data_dir /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print --project scPrinter_seq_PBMC_bulkATAC --enable_wandb

In [ ]:
!CUDA_VISIBLE_DEVICES=0 seq2print_train --config /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print/configs/PBMC_bulkATAC_Tcell_1_fold0.JSON --temp_dir /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print/temp --model_dir /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print/model --data_dir /net/talisker/home/benos/mae117/Documents/careers/opalia/the-milk-man/seq2print --project scPrinter_seq_PBMC_bulkATAC --enable_wandb